# Ass 2

##### setup

In [2]:
import pandas as pd
import os
import numpy as np
import statsmodels
print(os.getcwd())
data = pd.read_csv('dest.csv', 
                header=None, 
                names=['age', 'age2', 'ed',
                       'exper', 'exper2', 'nearc2',
                       'nearc4', 'nearc4a', 'nearc4b',
                       'race', 'smsa', 'south', 'wage'])
print(data.columns.values)

C:\Users\Samuel\OneDrive\UvA\S01P03\Advanced Econometrics II\Assignments
['age' 'age2' 'ed' 'exper' 'exper2' 'nearc2' 'nearc4' 'nearc4a' 'nearc4b'
 'race' 'smsa' 'south' 'wage']


## question 1

In [8]:
#create data
rho_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
a_list = [1, 0.6, 0.3, 0.15, 0.07, 0.04, 0.02, 0]
np.random.seed(1211)
#rho_list = [0, 0.1]
#a_list = [1, 0.6]

N = 100
k = 10
Z = np.matrix(np.random.normal(loc=0, scale=1, size=(N, k)))
beta_0 = 0
MC = 150

results = np.zeros([len(rho_list), len(a_list)])

for a in a_list:
    for rho in rho_list:
        
        rejection = np.zeros([MC, 1])
        
        for iter in range(MC):
            pi = np.matrix(np.zeros([k, 1]))
            pi[0, 0] = a
            sigma = np.array([[1, rho], [rho, 1]])
            eps_v = np.matrix(np.random.multivariate_normal(mean = [0,0], cov = sigma, size = ([N])))
            eps = eps_v[:,0]
            v = eps_v[:,1]

            X = Z @ pi + v
            Y = X * beta_0 + eps
            Pz = Z @ np.linalg.inv(Z.T @ Z) @ Z.T


            beta_2SLS = (X.T @ Pz @ X) ** (-1) @ (X.T @ Pz @ Y)
            residuals = Y - X * beta_2SLS

            S_hat = np.matrix(np.zeros([k, k]))

            for i in range(N):
                S_hat += (1 / N) * (residuals[i, 0] ** 2) * Z[i, :].T @ Z[i, :]

            beta_2SLS_var = N * (X.T @ Pz @ X) ** (-1) * \
                            (X.T @ Z @ np.linalg.inv(Z.T @ Z) @ S_hat @ np.linalg.inv(Z.T @ Z) @ Z.T @ X) * \
                            (X.T @ Pz @ X) ** (-1)

            t_stat = (beta_2SLS - beta_0) / np.sqrt(beta_2SLS_var)
            
            rejection[iter] = (t_stat > 1.96) | (t_stat < -1.96)
            
        results[rho_list.index(rho), a_list.index(a)] = round(np.mean(rejection), 3)
            
            


In [9]:
np.random.seed(1211)
rho_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
a_list = [1, 0.6, 0.3, 0.15, 0.07, 0.04, 0.02, 0]
N = 100
k = 10
MC = 150
Z = np.random.normal(loc = 0, scale = 1, size =(N,k))
beta_0 = 0
results_MC = np.zeros(MC)
R_freq = np.zeros((11,8))
for a in a_list:
    for rho in rho_list:
        for l in range(MC):
            pi = np.zeros(k)
            pi[0] = a
            sigma = np.array([[1, rho], [rho, 1]])
            eps_v = np.random.multivariate_normal(mean = [0,0], cov = sigma, size = (N))
            eps = eps_v[:,0]
            v = eps_v[:,1]

            X = Z @ pi + v
            Y = X * beta_0 + eps
            Pz = Z @ np.linalg.inv(Z.T @ Z) @ Z.T


            beta_2SLS = (X.T @ Pz @ X) ** (-1) * (X.T @ Pz @ Y)
            residuals = Y - X.T * beta_2SLS
            Shat = np.zeros((10,10))
            for i in range(Z.shape[1]):
                Shat += N ** (-1) * residuals[i] ** 2 * (np.outer(Z[i,], Z[i,]))


            beta_2SLS_var = N * (X.T @ Pz @ X) ** (-1) * \
                            (X.T @ Z @ np.linalg.inv(Z.T @ Z) @ Shat @ np.linalg.inv(Z.T @ Z) @ Z.T @ X) * \
                            (X.T @ Pz @ X) ** (-1)

            tstat = (beta_2SLS - beta_0)/np.sqrt(beta_2SLS_var)
            results_MC[l] = (tstat < -1.96) | (tstat > 1.96)
        R_freq[rho_list.index(rho), a_list.index(a)] = round(np.mean(results_MC),3)
            

In [10]:
print(results)
print(R_freq)

[[ 0.113  0.04   0.053  0.033  0.027  0.007  0.027  0.013]
 [ 0.053  0.04   0.073  0.047  0.04   0.033  0.04   0.04 ]
 [ 0.053  0.053  0.047  0.1    0.107  0.04   0.047  0.113]
 [ 0.087  0.06   0.06   0.127  0.207  0.173  0.14   0.247]
 [ 0.1    0.067  0.14   0.247  0.287  0.267  0.26   0.273]
 [ 0.1    0.133  0.28   0.393  0.427  0.387  0.473  0.48 ]
 [ 0.107  0.18   0.413  0.5    0.613  0.653  0.7    0.573]
 [ 0.08   0.167  0.46   0.653  0.8    0.807  0.793  0.767]
 [ 0.113  0.3    0.647  0.833  0.92   0.927  0.953  0.92 ]
 [ 0.2    0.327  0.687  0.953  0.993  0.993  0.993  1.   ]
 [ 0.207  0.34   0.8    0.967  1.     1.     1.     1.   ]]
[[ 0.553  0.527  0.64   0.567  0.6    0.56   0.6    0.533]
 [ 0.52   0.5    0.533  0.56   0.62   0.613  0.6    0.58 ]
 [ 0.547  0.6    0.607  0.613  0.62   0.593  0.613  0.633]
 [ 0.567  0.66   0.573  0.66   0.747  0.727  0.68   0.687]
 [ 0.56   0.593  0.68   0.7    0.78   0.82   0.8    0.853]
 [ 0.607  0.633  0.667  0.82   0.867  0.833  0.887  0.9